In [9]:
# Resolve paths from root project directory

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd, numpy
import pickle

def load_chunks(directory):
    chunk_dfs = []
    for name in sorted(os.listdir(directory)):
        sub_directory = os.path.join(directory, name)
        if os.path.isdir(sub_directory):
            for filename in sorted(os.listdir(sub_directory)):
                with open(os.path.join(sub_directory, filename), "r") as f:
                    chunk = pd.read_csv(f)
                    chunk_dfs.append(chunk)
        elif name.endswith(".csv"):
            chunk = pd.read_csv(os.path.join(directory, name))
            chunk_dfs.append(chunk)

    return pd.concat(chunk_dfs)

In [3]:
DATE = "16-dec"
DATA_DIR = "../data/parsed-export/{}/".format(DATE)
EXPORT_DIR = "../data/dataframes/{}/".format(DATE)

In [72]:
#df_retweets = pd.read_pickle(EXPORT_DIR + 'df_retweets.pickle')
df_users = pd.read_pickle(EXPORT_DIR + 'df_users.pickle')
#df_recent_tweets = pd.read_pickle(EXPORT_DIR + 'df_recent_tweets.pickle')

NameError: name 'EXPORT_DIR' is not defined

In [6]:
df_users.to_pickle("./df_users.pickle")

In [5]:
df_users.shape

(1388621, 12)

In [5]:
import json
with open("./data_export/url_stats/16-dec/expanded_url_map.json") as json_file:
    expanded_url_map = json.load(json_file)

In [6]:
def expand_url(url):
    expanded_url_data = expanded_url_map[url]["expanded_url"]
    expanded_url = expanded_url_data["url"] if "url" in expanded_url_data else url
    return expanded_url

In [7]:
df_recent_tweets["expanded_urls"] = df_recent_tweets.apply(lambda x: [expand_url(url) for url in x["urls"] if "twitter.com/" not in url], axis=1)

In [19]:
df_recent_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7603103 entries, 0 to 7609004
Data columns (total 48 columns):
 #   Column                      Dtype           
---  ------                      -----           
 0   datastore_id                object          
 1   quote_tweet                 object          
 2   hashtags                    object          
 3   retweet_count               int32           
 4   urls                        object          
 5   hasMedia                    bool            
 6   quote_count                 int32           
 7   timestamp                   object          
 8   text                        object          
 9   user                        object          
 10  tokens                      object          
 11  #electionfraud              Sparse[int32, 0]
 12  election fraud              Sparse[int32, 0]
 13  #stopthesteal               Sparse[int32, 0]
 14  voter fraud                 Sparse[int32, 0]
 15  election tampering          Spar

In [10]:
df_recent_tweets = df_recent_tweets[["datastore_id", "user", "hashed_user_id", "quote_tweet", "hashtags", "hasMedia", "retweet_count", "quote_count", "timestamp", "text", "urls", "expanded_urls"]]
df_recent_tweets.to_pickle("df_recent_tweets_subset_with_expanded_urls.pickle")

In [11]:
df_recent_tweets = pd.read_pickle("df_recent_tweets_subset_with_expanded_urls.pickle")

In [2]:
import hashlib, uuid

# Generate salt
# salt = uuid.uuid4().hex
# Read salt from file
with open("../private/anonymization-salt", "r") as f:
    salt = f.readline()

def anonymize_str(string):
    h = hashlib.sha256(string.encode() + salt.encode())
    return h.hexdigest()

In [9]:
df_recent_tweets["hashed_user_id"] = df_recent_tweets.apply(lambda t:anonymize_str(t["user"]), axis=1)

In [10]:
df_recent_tweets.head()

NameError: name 'df_recent_tweets' is not defined

In [7]:
df_retweets["hashed_user_id"] = df_retweets.apply(lambda t:anonymize_str(t["user"]), axis=1)
df_retweets["hashed_retweeted_from_user_id"] = df_retweets.apply(lambda t:anonymize_str(t["retweetedFrom_user"]), axis=1)

In [8]:
df_retweets.to_pickle("df_retweets_with_hashed_users.pickle")

In [76]:
import json
user_ids = set() 
users = []
duplicates = 0
for fname in ["users_retweets_2.jsonl", "users_retweets_3.jsonl", "users_retweets.jsonl"]:
    with open ("../data/fetch_users/" + fname, "r") as f:
        for line in f.readlines():
            user = json.loads(line)
            if (user["id"] not in user_ids):
                users.append({
                    "datastore_id": str(user["id"]),
                    "name": user["name"],
                    "handle": user["screen_name"],
                    "created_at": user["name"],
                    "verified": user["verified"],
                    "description": user["description"],
                    "followers_count": user["followers_count"],
                    "location": user["location"],
                    "created_at": user["created_at"],
                })
                user_ids.add(user["id"])
            else:
                duplicates += 1

print("Dups", duplicates)
print("Users", len(users))

Dups 18382
Users 1071554


In [77]:
df_users_retweets = pd.DataFrame(users)

In [78]:
df_users = pd.read_pickle("df_users.pickle")

In [81]:
existing_user_ids = set(df_users.index)

In [82]:
print(len(existing_user_ids.union(user_ids)))

2460175


In [86]:
df_users_retweets.to_pickle("./df_users_retweets.pickle")

In [21]:
df_published_tweets.head()

,user_community,user_active_status,retweet_count_metadata,quote_count_metadata,retweet_count_by_community_0,quote_count_by_community_0,retweet_count_by_community_1,quote_count_by_community_1,retweet_count_by_community_2,quote_count_by_community_2,retweet_count_by_community_3,quote_count_by_community_3,retweet_count_by_community_4,quote_count_by_community_4,retweet_count_by_suspended_users,quote_count_by_suspended_users
tweet_id,,,,,,,,,,,,,,,,
1319685030826938369,2.0,deleted,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1319685039714717696,NaN,active,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1319685089375158274,NaN,active,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1319685151031500801,NaN,suspended,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1319685176436396035,2.0,deleted,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
df_published_tweets = load_chunks("../data/published/tweets/").astype({"tweet_id": "str"}).set_index("tweet_id")
df_published_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7603103 entries, 1319685030826938369 to 1339195780704382980
Data columns (total 16 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   user_community                    float64
 1   user_active_status                object 
 2   retweet_count_metadata            int64  
 3   quote_count_metadata              int64  
 4   retweet_count_by_community_0      int64  
 5   quote_count_by_community_0        int64  
 6   retweet_count_by_community_1      int64  
 7   quote_count_by_community_1        int64  
 8   retweet_count_by_community_2      int64  
 9   quote_count_by_community_2        int64  
 10  retweet_count_by_community_3      int64  
 11  quote_count_by_community_3        int64  
 12  retweet_count_by_community_4      int64  
 13  quote_count_by_community_4        int64  
 14  retweet_count_by_suspended_users  int64  
 15  quote_count_by_suspended_users    int64  
dtypes: float64(

In [43]:
df_tweets_joined = df_published_tweets.join(df_recent_tweets.rename(columns={"datastore_id": "tweet_id"}).astype({"tweet_id": "str"}).set_index("tweet_id"), how="inner")

In [44]:
df_tweets_joined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7603103 entries, 1319685030826938369 to 1339195780704382980
Data columns (total 27 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   user_community                    float64
 1   user_active_status                object 
 2   retweet_count_metadata            int64  
 3   quote_count_metadata              int64  
 4   retweet_count_by_community_0      int64  
 5   quote_count_by_community_0        int64  
 6   retweet_count_by_community_1      int64  
 7   quote_count_by_community_1        int64  
 8   retweet_count_by_community_2      int64  
 9   quote_count_by_community_2        int64  
 10  retweet_count_by_community_3      int64  
 11  quote_count_by_community_3        int64  
 12  retweet_count_by_community_4      int64  
 13  quote_count_by_community_4        int64  
 14  retweet_count_by_suspended_users  int64  
 15  quote_count_by_suspended_users    int64  
 16  user       

In [45]:
df_tweets_joined.head()

,user_community,user_active_status,retweet_count_metadata,quote_count_metadata,retweet_count_by_community_0,quote_count_by_community_0,retweet_count_by_community_1,quote_count_by_community_1,retweet_count_by_community_2,quote_count_by_community_2,...,hashed_user_id,quote_tweet,hashtags,hasMedia,retweet_count,quote_count,timestamp,text,urls,expanded_urls
tweet_id,,,,,,,,,,,,,,,,,,,,,
1319685030826938369,2.0,deleted,0,0,0,0,0,0,0,0,...,3f65b87e850f0e814eacfe13f0f39c94ed61d4b6ea170d...,1317209207045820416,[VoterFraud],False,0,0,2020-10-23T17:00:04Z,This can’t be true - Voting “expert” &amp; Lib...,[https://twitter.com/i/web/status/131968503082...,[]
1319685039714717696,NaN,active,0,0,0,0,0,0,0,0,...,e456f60433562381b2b7dd60f3777aa9c7947b2861c5b2...,None,[],False,0,0,2020-10-23T17:00:06Z,U.S. Attorney Scott W. Brady has appointed two...,[https://twitter.com/i/web/status/131968503971...,[]
1319685089375158274,NaN,active,0,0,0,0,0,0,0,0,...,7cd7abda63b7a37d1d968ae84bc78c2c1eaa04a136f9e8...,None,[],False,0,0,2020-10-23T17:00:18Z,"@KrisKobach1787 Kris, I applaud your trying to...",[https://twitter.com/i/web/status/131968508937...,[]
1319685151031500801,NaN,suspended,0,0,0,0,0,0,0,0,...,f046a4a29c697d78bb89fcf256b44dca02015fb490140c...,None,[],True,0,0,2020-10-23T17:00:33Z,@IlhanMN And your actions like voter fraud sho...,[],[]
1319685176436396035,2.0,deleted,0,0,0,0,0,0,0,0,...,3f65b87e850f0e814eacfe13f0f39c94ed61d4b6ea170d...,1316835915369787392,[VoterFraud],False,0,0,2020-10-23T17:00:39Z,This can’t be true - Voting “expert” &amp; Lib...,[https://twitter.com/i/web/status/131968517643...,[]


In [46]:
df_tweets_joined.to_pickle("./df_tweets_joined.pickle")

In [87]:
df_users_retweets = pd.read_pickle("df_users_retweets.pickle").set_index("datastore_id")
df_users = pd.read_pickle("df_users.pickle")[df_users_retweets.columns]
df_users.head()

,name,handle,created_at,verified,description,followers_count,location
datastore_id,,,,,,,
1001056184256876546,eric reisinger,ericreisinger6,2018-05-28T11:02:47Z,False,None,14,"Boonville, MO"
1001058496463474688,AName,2865428T,2018-05-28T11:11:59Z,False,None,9,None
1001059579730579457,Justice is Coming⭐️⭐️⭐️,RedPilledisBACK,2018-05-28T11:16:17Z,False,#Trump2020 #17 #DarktoLight #JusticeLeague #We...,18383,None
1001061067286556672,AP23871,AP23871,2018-05-28T11:22:12Z,False,None,1,"Chicago, IL"
1001061562772209664,🦠🇨🇦Gas Station Sushi🇨🇦🦠,H0oFHearted,2018-05-28T11:24:10Z,False,#COVID-19Negative,816,Trudeau Tax Hell


In [88]:
df_users_retweets.head()

,name,handle,created_at,verified,description,followers_count,location
datastore_id,,,,,,,
393232103,Dan Lagnöhed,dlagnohed,Tue Oct 18 06:53:44 +0000 2011,False,,66,
4839391234,dag,dag77867675,Sat Jan 23 18:23:54 +0000 2016,False,,124,
1224501353831108609,zach,zach2676,Tue Feb 04 01:14:17 +0000 2020,False,,17,MI
1237745867731480576,Gomariana1202,gomariana1202,Wed Mar 11 14:23:33 +0000 2020,False,servir,25,
695809445755035648,Blazikus @HOLDING,BlazikusSmash,Sat Feb 06 03:21:14 +0000 2016,False,diddy 4 lyfe,178,"Chalfont, PA Los Angeles, CA"


In [89]:
df_users_joined = pd.concat([df_users, df_users_retweets]).reset_index().rename(columns={"datastore_id": "user_id"}).sort_values("created_at").set_index("user_id")
print(df_users_joined.shape)
df_users_joined.head()

(2460175, 7)


,name,handle,created_at,verified,description,followers_count,location
user_id,,,,,,,
1149389372824924160,,chunyan20817389,1970-01-01T00:00:00Z,False,None,84,None
1333952375166431232,,dredragunovska,1970-01-01T00:00:00Z,False,None,2,None
1268957229220233217,rizeyunggawd,rizeyunggawd,1970-01-01T00:00:00Z,False,right vs wrong\nI am a black man. Don't be an ...,63,None
22,rabble,rabble,2006-03-21T21:04:12Z,False,"anarchist, hacker, troublemaker (he/him) worki...",16724,from PDX to MVD
224,Dave Pell,davepell,2006-05-01T22:17:42Z,True,"Managing Editor, Internet.",57532,San Francisco


In [61]:
df_users_joined[~df_users_joined.index.duplicated(keep='first')].shape

(2460175, 7)

In [91]:
df_published_users = load_chunks("../data/published/users/").astype({"user_id": "str"}).set_index("user_id")
df_published_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2559018 entries, 6590 to 1269104794511147008
Data columns (total 16 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   user_community                          float64
 1   user_active_status                      object 
 2   closeness_centrality_detractor_cluster  float64
 3   closeness_centrality_promoter_cluster   float64
 4   retweet_count_by_community_0            int64  
 5   quote_count_by_community_0              int64  
 6   retweet_count_by_community_1            int64  
 7   quote_count_by_community_1              int64  
 8   retweet_count_by_community_2            int64  
 9   quote_count_by_community_2              int64  
 10  retweet_count_by_community_3            int64  
 11  quote_count_by_community_3              int64  
 12  retweet_count_by_community_4            int64  
 13  quote_count_by_community_4              int64  
 14  retweet_count_by_suspend

In [92]:
df_users_all_fields = df_published_users.join(df_users_joined)
print(df_users_all_fields.shape)
print(df_users_all_fields.name.isnull().sum())

(2559018, 23)
98843


In [93]:
df_users_all_fields[df_users_all_fields.name.isnull()]

,user_community,user_active_status,closeness_centrality_detractor_cluster,closeness_centrality_promoter_cluster,retweet_count_by_community_0,quote_count_by_community_0,retweet_count_by_community_1,quote_count_by_community_1,retweet_count_by_community_2,quote_count_by_community_2,...,quote_count_by_community_4,retweet_count_by_suspended_users,quote_count_by_suspended_users,name,handle,created_at,verified,description,followers_count,location
user_id,,,,,,,,,,,,,,,,,,,,,
715718947371229184,2.0,active,NaN,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275376998,3.0,active,NaN,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275429414,3.0,active,NaN,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275447451,1.0,active,NaN,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275501620,1.0,active,NaN,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899317849243693057,2.0,suspended,NaN,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1296302672077758464,2.0,suspended,NaN,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1403510982,1.0,deleted,NaN,NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
df_users_all_fields[df_users_all_fields.name.isnull()].value_counts("user_active_status")

user_active_status
deleted      39577
active       31872
suspended    27394
dtype: int64

In [23]:
df_published_users.head()

,user_community,user_active_status,closeness_centrality_detractor_cluster,closeness_centrality_promoter_cluster,retweet_count_by_community_0,quote_count_by_community_0,retweet_count_by_community_1,quote_count_by_community_1,retweet_count_by_community_2,quote_count_by_community_2,retweet_count_by_community_3,quote_count_by_community_3,retweet_count_by_community_4,quote_count_by_community_4,retweet_count_by_suspended_users,quote_count_by_suspended_users
user_id,,,,,,,,,,,,,,,,
6590,0.0,suspended,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
10597,NaN,deleted,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
16943,NaN,suspended,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
29803,NaN,suspended,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
68093,0.0,suspended,0.076194,NaN,16,0,0,0,0,0,0,0,0,0,1,0


In [57]:
df_users_all_fields.head()

,user_community,user_active_status,closeness_centrality_detractor_cluster,closeness_centrality_promoter_cluster,retweet_count_by_community_0,quote_count_by_community_0,retweet_count_by_community_1,quote_count_by_community_1,retweet_count_by_community_2,quote_count_by_community_2,...,quote_count_by_community_4,retweet_count_by_suspended_users,quote_count_by_suspended_users,name,handle,created_at,verified,description,followers_count,location
user_id,,,,,,,,,,,,,,,,,,,,,
6590,0.0,suspended,NaN,NaN,0,0,0,0,0,0,...,0,0,0,CyberBenB,cyberbenb,2006-09-21T18:27:47Z,False,He/They - OKC/Spatz cult Survivor | info activ...,3527.0,🖤❤️💛🐧🏴‍☠️😷🏳️‍🌈☯️♻️🌍📢
10597,NaN,deleted,NaN,NaN,0,0,0,0,0,0,...,0,0,0,Ferdinand Reinke,reinkefj,2006-10-26T14:18:36Z,False,A fat old white guy injineer who is in pursuit...,52.0,08824
16943,NaN,suspended,NaN,NaN,0,0,0,0,0,0,...,0,0,0,Drew Robinson,ocean,2006-11-23T03:21:03Z,False,Relentless pursuer of Quality. Full stack deve...,1320.0,"Perth, WA, Australia"
29803,NaN,suspended,NaN,NaN,0,0,0,0,0,0,...,0,0,0,Nichelle “On Neida and Mildred” Stephens,niche,2006-11-29T06:57:41Z,False,Trust Black Women. \nEmail me for press inqu...,8563.0,"Savannah, GA"
68093,0.0,suspended,0.076194,NaN,16,0,0,0,0,0,...,0,1,0,Margery,margery,2006-12-14T18:41:19Z,False,Carpe Diem. Despise and loathe Trump #Resister,1696.0,"Massachusetts, USA"


In [95]:
df_users_all_fields.to_pickle("df_users_all_fields.pickle")

In [65]:
published_user_ids = set(df_published_users.index)
len(published_user_ids)

2559018

In [68]:
known_user_ids = set(df_users_joined.index)
len(known_user_ids)

2460175

In [71]:
df_tweets.head()

NameError: name 'df_tweets' is not defined